In [45]:
import pandas as pd
import numpy as np

In [46]:
path = "C:/Users/user/Documents/GitHub/Visual-Analytics/dataset/"
ctg = pd.read_csv(path+"prod_cat_info.csv") #categories csv
cst = pd.read_csv(path+"Customer.csv")
tr = pd.read_csv(path+"Transactions.csv")

In [47]:
ctg.head()

,prod_cat_code,prod_cat,prod_sub_cat_code,prod_subcat
0,1,Clothing,4,Mens
1,1,Clothing,1,Women
2,1,Clothing,3,Kids
3,2,Footwear,1,Mens
4,2,Footwear,3,Women


In [48]:
cst.head()

,customer_Id,DOB,Gender,city_code
0,268408,02-01-1970,M,4.0
1,269696,07-01-1970,F,8.0
2,268159,08-01-1970,F,8.0
3,270181,10-01-1970,F,2.0
4,268073,11-01-1970,M,1.0


In [49]:
tr.head()

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
0,80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,e-Shop
1,29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,e-Shop
2,51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop
3,93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,e-Shop
4,51750724947,273420,23-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop


In [50]:
tr['Qty'] = np.abs(tr['Qty'])
tr['total_amt'] = np.abs(tr['total_amt'])
tr.head()

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
0,80712190438,270351,28-02-2014,1,1,5,-772,405.300,4265.300,e-Shop
1,29258453508,270384,27-02-2014,5,3,5,-1497,785.925,8270.925,e-Shop
2,51750724947,273420,24-02-2014,6,5,2,-791,166.110,1748.110,TeleShop
3,93274880719,271509,24-02-2014,11,6,3,-1363,429.345,4518.345,e-Shop
4,51750724947,273420,23-02-2014,6,5,2,-791,166.110,1748.110,TeleShop


In [51]:
#Merge transaction data with category information
tr_merged = pd.merge(tr, ctg,  how='left', left_on=['prod_cat_code','prod_subcat_code'], 
right_on = ['prod_cat_code','prod_sub_cat_code'])

#Merge with customer information
tr_final = pd.merge(tr_merged, cst,  how='left', left_on=['cust_id'], 
right_on = ['customer_Id'])
tr_final = tr_final.sort_values(by=["tran_date"])

tr_final.to_csv(path+'full_data.csv', index=False)
tr_final.head()

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type,prod_cat,prod_sub_cat_code,prod_subcat,customer_Id,DOB,Gender,city_code
16050,47933752822,270192,1/1/2012,3,1,4,215,90.300,950.300,e-Shop,Clothing,3,Kids,270192,01-06-1987,F,4.0
16052,35798683065,271913,1/1/2012,1,1,1,229,24.045,253.045,MBR,Clothing,1,Women,271913,15-10-1985,F,2.0
16053,30078154552,269151,1/1/2012,3,2,1,456,47.880,503.880,e-Shop,Footwear,3,Women,269151,29-04-1981,F,2.0
16054,57039573748,268282,1/1/2012,4,1,1,1024,107.520,1131.520,MBR,Clothing,4,Mens,268282,22-12-1970,F,1.0
16055,41901986246,269269,1/1/2012,7,5,2,1122,235.620,2479.620,MBR,Books,7,Fiction,269269,04-02-1981,M,1.0


In [20]:
grouped_tr = tr.groupby(['prod_subcat_code', 'prod_cat_code'])["Qty", "total_amt"].apply(lambda x : x.astype(int).sum())
grouped_tr.head()

Qty  total_amt
prod_subcat_code prod_cat_code                 
1                1              3085    2677312
                 2              2814    2439523
                 4              2976    2623618
2                6              3073    2651658
3                1              2961    2644253

In [28]:
merged_tr = pd.merge(grouped_tr, ctg,  how='outer', left_on=['prod_cat_code','prod_subcat_code'], 
right_on = ['prod_cat_code','prod_sub_cat_code'])
merged_tr = merged_tr.sort_values(by=["prod_cat_code","prod_sub_cat_code"])
merged_tr.head()

,prod_cat_code,Qty,total_amt,prod_cat,prod_sub_cat_code,prod_subcat
0,1,3085,2677312,Clothing,1,Women
4,1,2961,2644253,Clothing,3,Kids
7,1,2916,2574840,Clothing,4,Mens
1,2,2814,2439523,Footwear,1,Mens
5,2,3075,2576409,Footwear,3,Women


Use this in js:

date = //read Python generated data here

var nestedData = d3.nest()
 .key(function(d) { return d.category; })
 .key(function(d) { return d.subcategory; })
 .entries(data);